In [1]:
#!pip install gradio  
#!pip install  transformers
#!pip install nltk



In [2]:

import gradio as gr
import re
import warnings
import pickle


#from gradio.mix import Parallel
from nltk.tokenize import sent_tokenize
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    Wav2Vec2ForCTC,
    Wav2Vec2Tokenizer,
    pipeline,
)


# Ignore specific warnings
warnings.filterwarnings("ignore")



**Load Data and Define Summarizing functions**

In [37]:

with open('C:/Users/njoki/OneDrive/Desktop/Capstone Project/cleaned_data.pkl', 'rb') as file:
     dataset = pickle.load(file)

In [38]:
len(dataset)

510

In [5]:
dataset[1]

'EXHIBIT  INFORMATION EXHIBIT IDENTIFIED CONFIDENTIAL EXCLUDED MATERIAL II WOULD LIKELY CAUSE COMPETITIVE HARM REGISTRANT PUBLICLY DISCLOSED. SERVICES AGREEMENT Services Agreement Agreement entered October   made effective November   Effective Date Provider TELCOSTAR PTE LTD. company organized existing laws Singapore Ability Computer Software Industries Ltd company organized existing laws State Israel Recipient. foregoing parties referred herein Party together Parties. RECITALS A. Recipient wishes engage Provider provide certain services resources Services Provider desires provide Recipient Services accordance terms conditions set forth herein. AGREEMENT Parties hereby agree follows . Services.  Provision Services. Provider agrees provide Services set forth Exhibit attached hereto Exhibit may amended supplemented pursuant terms Agreement Exhibit Recipient respective periods terms conditions set forth Agreement Exhibit. Notwithstanding contents Exhibit Provider agrees respond good faith

**a. Summarization via Hugging Face Pipeline**


Hugging Face's pipeline allows you to load up several summarization models, from FB's Bart to Google's T5. 

In [6]:

#create a pipeline

pipeline_summ = pipeline(
    "summarization",
    model="facebook/bart-large-cnn", # one can also use"t5-small" etc 
    tokenizer="facebook/bart-large-cnn",
    framework="pt",
)

# First of 2 summarization function
def fb_summarizer(dataset):
    #input_text = dataset
    results = pipeline_summ(dataset)
    return [result["summary_text"] for result in results]



In [7]:
# First of 2 Gradio apps to use in "parallel"
summary1 = gr.Interface(
    fn=fb_summarizer,
    inputs=gr.Textbox(),
    outputs=gr.Textbox(label="Summary by FB/Bart-large"),
)

summary1.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://dc023118950fce7a8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**b. Summarization using Hugging  Face's Model Hub**

In [8]:
model_name = "google/pegasus-cnn_dailymail" 

# Second of 2 summarization function
def google_summarizer(dataset):
    input_texts = dataset
    
    tokenizer_pegasus = AutoTokenizer.from_pretrained(model_name)

    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    batch = tokenizer_pegasus.prepare_seq2seq_batch(
        input_texts, truncation=True, padding="longest", return_tensors="pt"
    )
    translated = model_pegasus.generate(**batch)

    pegasus_summary = tokenizer_pegasus.batch_decode(
        translated, skip_special_tokens=True
    )

    return pegasus_summary[0]




In [9]:
summary2 = google_summarizer(dataset[1])
print(summary2)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TELCOSTAR PTE LTD. company organized existing laws Singapore Ability Computer Software Industries Ltd company organized existing laws State Israel.<n>Service Agreement entered October made effective November Effective Date Provider TELCOSTAR PTE LTD. company organized existing laws Singapore Ability Computer Software Industries Ltd company organized existing laws State Israel.


In [10]:
# Second of 2 Gradio apps to use in "parallel"
summary2 = gr.Interface(
    fn=google_summarizer,
    inputs=gr.Textbox(),
    outputs=gr.Textbox(label="Summary by Google/Pegasus-CNN-Dailymail"),
)

summary2.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a448127755a949fda5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**2. Launch the Models in Parallel**

In [11]:
!pip install sentencepiece

  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)


In [39]:
# Create a Gradio Interface with parallel inputs
def parallel_summaries(text):
    # Call individual summarization functions
    fb_summary = fb_summarizer(text)
    google_summary = google_summarizer(text)
    return fb_summary, google_summary

iface = gr.Interface(
    fn=parallel_summaries,  # List of functions for parallel processing
    inputs=[gr.Textbox(lines=20, label="Paste some text here"),
            gr.File(label="Or Upload a Text File"),
            gr.Dropdown(choices=dataset)],
    outputs=[gr.Textbox(interactive=False, label="Summary 1"), 
             gr.Textbox(interactive=False, label="Summary 2")],
    title="Compare 2 AI Summarizers"
)

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://857deefaa1a21781c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [40]:
gradio deploy

SyntaxError: invalid syntax (3470978507.py, line 1)

Traceback (most recent call last):
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 489, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-p